In [46]:
import math
import math
import operator
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,r2_score,mean_absolute_error,mean_squared_error
from sklearn.tree import DecisionTreeClassifier
eps = np.finfo(float).eps
from sklearn import linear_model

In [47]:
#Assigning column names to data
df=pd.read_csv('data.csv')
df=df.drop('Serial No.',axis=1)
columns=df.columns
len(df)
for i in range(len(df)):
    if df.iloc[i,-1] >= 0.5:
        df.iloc[i,-1]=1
    else:
        df.iloc[i,-1]=0

In [48]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]
train_k=train
test_k=test
test_k

GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  Research  \
1          319          108                  3  3.0   3.5  8.54         1   
3          326          113                  5  4.5   4.0  9.40         1   
8          315          105                  3  2.0   2.5  8.48         0   
9          296           97                  2  1.5   2.0  7.80         0   
13         321          109                  3  3.0   3.0  8.54         1   
14         333          119                  5  5.0   4.5  9.78         1   
16         332          118                  2  4.5   3.5  9.36         1   
17         324          100                  3  4.0   5.0  8.64         1   
19         299          100                  2  2.0   2.0  7.88         0   
27         310          105                  2  3.0   3.5  8.01         0   
29         331          115                  5  4.0   3.5  9.44         1   
32         318          109                  3  3.0   3.0  8.50         0   
33         327          109                  3  3.5   4.0  8.77         1   
34         314          104                  4  5.0   5.0  9.02         0   
35         315          105                  2  2.0   2.5  7.65         0   
37         301           99                  2  3.0   2.0  8.22         0   
42         312          105                  3  2.0   3.0  8.02         1   
45         298           98                  2  4.0   3.0  8.03         0   
53         312          104                  3  3.5   4.0  8.09         0   
66         327          111                  4  3.0   4.0  8.40         1   
69         311          106                  2  3.5   3.0  8.26         1   
72         308          108                  4  4.5   5.0  8.34         0   
81         311          104                  3  4.0   3.5  8.13         1   
89         326          116                  2  4.5   3.0  9.08         1   
90         321          114                  4  4.0   5.0  9.12         0   
92         300          100                  3  2.0   3.0  8.66         1   
95         340          120                  4  4.5   4.0  9.92         1   
99         298           98                  2  1.5   2.5  7.50         1   
103        296           95                  2  3.0   2.0  7.54         1   
110        314          109                  4  3.5   4.0  8.77         1   
..         ...          ...                ...  ...   ...   ...       ...   
363        314          105                  2  2.5   2.0  7.64         0   
365        322          103                  4  3.0   2.5  8.02         1   
370        313          107                  2  2.5   2.0  8.50         1   
371        316          111                  4  4.0   5.0  8.54         0   
373        331          119                  4  5.0   4.5  9.34         1   
378        321          110                  3  3.5   5.0  8.85         1   
383        311          104                  2  2.0   2.0  8.30         0   
391        306          103                  2  2.5   3.0  8.36         0   
392        321          111                  4  4.0   4.0  8.97         1   
395        326          116                  3  3.5   4.0  9.14         1   
404        325          111                  4  4.0   4.5  9.11         1   
405        334          116                  4  4.0   3.5  9.54         1   
407        318          101                  5  3.5   5.0  8.78         1   
409        331          115                  5  4.5   3.5  9.36         1   
410        313          102                  3  2.0   3.0  8.27         0   
412        304          107                  3  3.5   3.0  7.86         0   
413        290          104                  4  2.0   2.5  7.46         0   
416        329          119                  4  4.5   4.5  9.16         1   
418        304           97                  2  1.5   2.0  7.64         0   
421        319          103                  3  2.5   4.0  8.76         1   
4

In [49]:
# train_X=train[:,:-1]
train_X=train.drop('Chance of Admit ',axis=1)
train_X=(train_X-train_X.mean())/train_X.std()
train_X.insert(0, 'x0', np.ones(len(train)))
train_Y=train['Chance of Admit ']
TX=train_X
TY=train_Y
m=train_X.shape[0]
train_Y=train_Y.values.reshape(m,1)
train_X=train_X.values

In [50]:
theta = np.zeros([8,1])
alpha = 0.01
iters = 1000

In [51]:
def computeCost(X,y,theta):
    h = np.dot(X ,theta)
    error = h-y
    loss = np.power(error,2)
    J = np.sum(loss)/(2*m)
    return J

In [52]:
def G(x):
    p=1+(np.exp(-x))
    return 1/p

In [53]:
def gradient_descent(X, y, theta, alpha, num_iters):
    costs = np.zeros(num_iters)
    for i in range(0,num_iters):
        gradient = G(np.matmul(X, theta))- y   
        theta = theta - (alpha/len(X))*np.sum(np.matmul(gradient.T,X))
        costs[i] = computeCost(X, y, theta)
    return theta
g = gradient_descent(train_X, train_Y, theta, alpha, iters)
#Print the results...
theta=g

In [54]:
def predict(theta,test_X):
    test_X.insert(0, 'x0', np.ones(len(test_X)))
    test_X=(test_X-train_X.mean())/test_X.std()
    pred=[]
    for row in range(len(test_X)):
        ans=0
#         print test_X.iloc[row,0],theta[0]
        for i in range(len(test_X.columns)):
            ans+=theta[i]*test_X.iloc[row,i]
        if ans >= 0.5 :
            pred.append(1)
        else:
            pred.append(0)
    return pred

In [55]:
theta=g

In [56]:
test_X=test.drop('Chance of Admit ',axis=1)
pred=predict(theta,test_X)
test_Y=test['Chance of Admit ']

In [57]:
accuracy_score(test_Y,pred)

0.8907563025210085

In [58]:
# # df=pd.read_csv('data.csv')
# msk = np.random.rand(len(df)) < 0.8
# train = df[msk]
# # test = df[~msk]
# train_X=train.drop('Chance of Admit ',axis=1)
# train_Y=train['Chance of Admit ']
# test_X=test.drop('Chance of Admit ',axis=1)
# test_Y=test['Chance of Admit ']
reg = linear_model.LogisticRegression(solver='lbfgs') 
# train the model using the training sets 
reg.fit(TX,TY)
pred2=reg.predict(test_X)
accuracy_score(test_Y,pred2)

0.8907563025210085

In [59]:
# finding the euclediandistance 
def euclediandistance(row1,row2,numberofattributes):
    distance=0
    for attribute in range(numberofattributes):
        distance+=pow(row1[attribute]-row2[attribute],2)
    distance=math.sqrt(distance)
    return distance
# euclediandistance([2,2,2,2,2],[1,1,4,4,3],4)

In [60]:
def findkneighbours(train,testrow,k):
    eucledianlist=[]
    for traininstance in range(len(train)):
        edistance=euclediandistance(train.iloc[traininstance],testrow,len(testrow)-1)
        eucledianlist.append((edistance,train.iloc[traininstance][-1]))
#     neighbourwithdistance=[]
#     neighbourwithdistane=findknearest(euclediandistance,k)
    eucledianlist=sorted(eucledianlist,key=operator.itemgetter(0))
    neighbour=[]
    for row in range(k):
        neighbour.append(eucledianlist[row][1])
    return neighbour

In [61]:
def predictinstance(neighbour):
    one=0
    zero=1
    for rows in range(len(neighbour)):
        if neighbour[rows] == 1:
            one=one+1
        elif neighbour[rows] == 0:
            zero=zero+1
    if one >= zero :
        return 1
    else :
        return 0

In [62]:
def predictnew(train,test,k):
    predictlist=[]
    for i in range(len(test)):
        neighbour=findkneighbours(train,test.iloc[i],k)
        predictlist.append(predictinstance(neighbour))
#     print predictlist
    return accuracy_score(test['Chance of Admit '],predictlist)

In [63]:
def predict(train,test,k):
    predictlist=[]
    for i in range(len(test)):
        neighbour=findkneighbours(train,test.iloc[i],k)
        predictlist.append(predictinstance(neighbour))
#     print predictlist
    print "The below details are for K-NN CLASSIFIER developed:-"
    print "Accuracy score:-"
    print accuracy_score(test['Chance of Admit '],predictlist)
    print "Confusion Matrix:-"
    print confusion_matrix(test['Chance of Admit '],predictlist)
    print "Classification Report:-"
    print classification_report(test['Chance of Admit '],predictlist)
    accuracy=[]
    for i in range(1, 50):
        p=predictnew(train,test,i)
        accuracy.append(p)    
    plt.plot(range(1, 50), accuracy)
    plt.title('Accuracy vs K Value')  
    plt.xlabel('K Value')
    plt.grid(True)
    plt.ylabel('Accuracy')
# print test['Class']    

In [ ]:
predict(train_k,test_k,5)

The below details are for K-NN CLASSIFIER developed:-
Accuracy score:-
0.9159663865546218
Confusion Matrix:-
[[  4   9]
 [  1 105]]
Classification Report:-
              precision    recall  f1-score   support

         0.0       0.80      0.31      0.44        13
         1.0       0.92      0.99      0.95       106

   micro avg       0.92      0.92      0.92       119
   macro avg       0.86      0.65      0.70       119
weighted avg       0.91      0.92      0.90       119

